## <u>KEVIN MUGO MWANIKI</u>
## <u>Chapter 8: Restructuring Data into a Tidy Form</u>
* [Tidying variable values as column names with stack](#Tidying-variable-values-as-column-names-with-stack)
* [Tidying variable values as column names with melt](#Tidying-variable-values-as-column-names-with-melt)
* [Stacking multiple groups of variables simultaneously](#Stacking-multiple-groups-of-variables-simultaneously)
* [Inverting stacked data](#Inverting-stacked-data)
* [Unstacking after a groupby aggregation](#Unstacking-after-a-groupby-aggregation)
* [Replicating pivot_table with a groupby aggregation](#Replicating-pivot_table-with-a-groupby-aggregation)
* [Renaming axis levels for easy reshaping](#Renaming-axis-levels-for-easy-reshaping)
* [Tidying when multiple variables are stored as column names](#Tidying-when-multiple-variables-are-stored-as-column-names)
* [Tidying when multiple variables are stored as column values](#Tidying-when-multiple-variables-are-stored-as-column-values)
* [Tidying when two or more values are stored in the same cell](#Tidying-when-two-or-more-values-are-stored-in-the-same-cell)
* [Tidying when variables are stored in column names and values](#Tidying-when-variables-are-stored-in-column-names-and-values)
* [Tidying when multiple observational units are stored in the same table](#Tidying-when-multiple-observational-units-are-stored-in-the-same-table)
    

In [1]:
pwd

'C:\\Users\\User\\Desktop\\PandasCookbook'

## <u>Introduction</u>
<p>It is necessary to structure data for further analysis so that the analysis and prediction can be done efficiently. This 
notebook will focus on Hadley Wickham procedure of organizing data into a format that is correct. The following three 
principles govern the creation of tidy data:</p>
<ul>
    <li>Each variable forms a column</li>
    <li>Each observation forms a row</li>
    <li>Each type of observational unit forms a table</li>
</ul>
<p><b>Variable names</b> are labels eg. Gender, race, salary, etc. <b>Variable values</b> are things that are liable to change
for every value eg. male/female for gender or white/black for race. An <b>observation</b> is the collection of all variable 
values for a single observation unit. Consider a retail store with data for its transactions, employees, customers, items and 
the store itself. Each of these can be considered as observational units and would require their own tables. Combining employee
and customer data can break the tidy data principle. One should understand where messy data exist and their boundless 
possibilities. According to Hadley Wickham, there are five main common types of messy data:</p>
<ul>
    <li>Column names are values, not variable names</li>
    <li>Multiple variables are stored in column names</li>
    <li>Variables are stored in both rows and columns</li>
    <li>Multiple types of observational units are stored in the same table</li>
    <li>A single observational unit is stored in multiple tables</li>
</ul>
<p>Tidy data neither involves change in dataset values, filling missing values or doing any form of analysis. It involves the
change in shape/structure of data to meet the tidy data principles. It makes all tools to be organized instead of being 
scattered randomly thus tasks can be completed easily. This even provides a better opportunity to carry out a deeper analysis. 
The main tidy tools that Pandas has for DataFrames are <b>stack, unstack, melt</b> and <b>pivot</b>. More complex tidying 
involves ripping apart text, which calls for the <b>str</b> accessor. Other helpful methods include: <b>rename, rename_axis, 
reset_index</b> and <b>set_index</b>. They are usually helpful in the final bits of tidying data</p>

## <u>PROBLEM 1: Tidying variable values as column names</u>
### <u>Option 1: Use of the stack  method</u>
<p>Transformation of horizontal column names into vertical column names is known as <b>melting, stacking</b> or <b>unpivoting</b>. Let us consider the following data that is not in tidy form. This is because <b>each column name is the value of a variable</b>. It is solved by identifying all variables. In the dataset, we have variables for <b>state</b> and <b>fruit</b>. There is also numeric data that was not identified anywhere in the context of the problem. We can label this variable as <b>weight</b> or any other sensible name:</p>

In [2]:
import pandas as pd
import numpy as np
state_fruit = pd.read_csv('data/state_fruit.csv', index_col = 0)

In [3]:
state_fruit

,Apple,Orange,Banana
Texas,12,10,40
Arizona,9,7,12
Florida,0,14,190


In [4]:
state_fruit.stack()

Texas    Apple      12
         Orange     10
         Banana     40
Arizona  Apple       9
         Orange      7
         Banana     12
Florida  Apple       0
         Orange     14
         Banana    190
dtype: int64

<p>We now have a Series in our MultiIndex. There are two levels in the index. The initial index has been pushed to the left to give way for an additional index which was the initial column. The <b>reset_index</b> method turns the result into a DataFrame and we have our tidy data:</p>

In [5]:
#Note how the indexing starts from zero
state_fruit_tidy = state_fruit.stack().reset_index()
state_fruit_tidy

,level_0,level_1,0
0,Texas,Apple,12
1,Texas,Orange,10
2,Texas,Banana,40
3,Arizona,Apple,9
4,Arizona,Orange,7
5,Arizona,Banana,12
6,Florida,Apple,0
7,Florida,Orange,14
8,Florida,Banana,190


In [87]:
#The last step is to make the column names meaningful:
#Method 1:
state_fruit_tidy.columns = ['state', 'fruit', 'weight']
state_fruit_tidy

,state,fruit,weight
0,Texas,Apple,12
1,Texas,Orange,10
2,Texas,Banana,40
3,Arizona,Apple,9
4,Arizona,Orange,7
5,Arizona,Banana,12
6,Florida,Apple,0
7,Florida,Orange,14
8,Florida,Banana,190


In [7]:
#Method 2: 
new_cols = {'level_0' : 'state', 'level_1' : 'fruit', '0' : 'weight'}
state_fruit_tidy = state_fruit_tidy.rename(columns = new_cols)
state_fruit_tidy

,state,fruit,weight
0,Texas,Apple,12
1,Texas,Orange,10
2,Texas,Banana,40
3,Arizona,Apple,9
4,Arizona,Orange,7
5,Arizona,Banana,12
6,Florida,Apple,0
7,Florida,Orange,14
8,Florida,Banana,190


In [8]:
#The second alternative to the above lines of code is:
state_fruit.stack()\
           .rename_axis(['state', 'fruit'])

state    fruit 
Texas    Apple      12
         Orange     10
         Banana     40
Arizona  Apple       9
         Orange      7
         Banana     12
Florida  Apple       0
         Orange     14
         Banana    190
dtype: int64

In [9]:
#We can add an extra method to the above code with the name parameter to reproduce a similar output to the above steps:
#The rename axis method takes the general initial names of the columns and rows
#The reset_index takes the name that will be used in the conversion of the Series to a DataFrame
sf = state_fruit.stack()\
           .rename_axis(['state', 'fruit'])\
           .reset_index(name = 'weight')

In [10]:
sf

,state,fruit,weight
0,Texas,Apple,12
1,Texas,Orange,10
2,Texas,Banana,40
3,Arizona,Apple,9
4,Arizona,Orange,7
5,Arizona,Banana,12
6,Florida,Apple,0
7,Florida,Orange,14
8,Florida,Banana,190


In [91]:
#One of the keys to use stack successfully is placing all of the columns that you did not wish to transform in the index
#The states were initially considered as the index. 
#If the states were not read in the index:
state_fruit2 = pd.read_csv('data/state_fruit.csv')

In [92]:
state_fruit2.stack()

0  Unnamed: 0      Texas
   Apple              12
   Orange             10
   Banana             40
1  Unnamed: 0    Arizona
   Apple               9
   Orange              7
   Banana             12
2  Unnamed: 0    Florida
   Apple               0
   Orange             14
   Banana            190
dtype: object

In [93]:
#The structure of this data is totally wrong:
state_fruit2.stack()\
            .rename_axis(['State', 'Fruits'])\
            .reset_index(name = 'Weight')

,State,Fruits,Weight
0,0,Unnamed: 0,Texas
1,0,Apple,12
2,0,Orange,10
3,0,Banana,40
4,1,Unnamed: 0,Arizona
5,1,Apple,9
6,1,Orange,7
7,1,Banana,12
8,2,Unnamed: 0,Florida
9,2,Apple,0


In [94]:
#In order to reshape this data correctly, you will need to put all the non-reshaped columns into the index first 
#The set_index method, and then use stack. 
#The following code gives a similar result to step 1:
state_fruit2.set_index('Unnamed: 0').stack()

Unnamed: 0        
Texas       Apple      12
            Orange     10
            Banana     40
Arizona     Apple       9
            Orange      7
            Banana     12
Florida     Apple       0
            Orange     14
            Banana    190
dtype: int64

In [95]:
#The code can then be finished up as follows:
sf2 = state_fruit2.set_index('Unnamed: 0')\
                  .stack()\
                  .rename_axis(['state', 'fruit'])\
                  .reset_index(name = 'weight')

In [96]:
sf2

,state,fruit,weight
0,Texas,Apple,12
1,Texas,Orange,10
2,Texas,Banana,40
3,Arizona,Apple,9
4,Arizona,Orange,7
5,Arizona,Banana,12
6,Florida,Apple,0
7,Florida,Orange,14
8,Florida,Banana,190


In [17]:
#Proof that the first method equals the second method
sf.equals(sf2)

True

### <u>Option 2: Use of the melt method</u>
<p>The <b>melt</b> method is more flexible than <b>stack</b> but they perform the same role. Pandas is pushing to move all
functions that operate on DataFrames to methods, such as how they did with <b>melt</b>. Here, we will use the melt method to 
tidy a simple dataframe with variable values and column names:</p>
<p>One crucial aspect of melt is that it ignores values in the index, and, in fact, it silently drops your index and replaces it with a default RangeIndex. This means that if you do have values in your index that you would like to keep, you will need to reset the index first before using melt.</p>

In [18]:
state_fruit2 = pd.read_csv('data/state_fruit2.csv')

In [19]:
state_fruit2

,State,Apple,Orange,Banana
0,Texas,12,10,40
1,Arizona,9,7,12
2,Florida,0,14,190


In [20]:
#Use of the melt method:
state_fruit2.melt(id_vars = ['State'], value_vars = ['Apple', 'Orange', 'Banana'])

,State,variable,value
0,Texas,Apple,12
1,Arizona,Apple,9
2,Florida,Apple,0
3,Texas,Orange,10
4,Arizona,Orange,7
5,Florida,Orange,14
6,Texas,Banana,40
7,Arizona,Banana,12
8,Florida,Banana,190


In [21]:
#To rename the last two columns:
sf_melt = state_fruit2.melt(id_vars = ['State'],
                            value_vars = ['Apple', 'Orange', 'Banana'],
                            var_name = 'Fruit',
                            value_name = 'Weight')
sf_melt

,State,Fruit,Weight
0,Texas,Apple,12
1,Arizona,Apple,9
2,Florida,Apple,0
3,Texas,Orange,10
4,Arizona,Orange,7
5,Florida,Orange,14
6,Texas,Banana,40
7,Arizona,Banana,12
8,Florida,Banana,190


In [22]:
#A shorter code can be written as follows:
#But customization is necessary to make it similar to that of cells above
state_fruit2.melt(id_vars='State')

,State,variable,value
0,Texas,Apple,12
1,Arizona,Apple,9
2,Florida,Apple,0
3,Texas,Orange,10
4,Arizona,Orange,7
5,Florida,Orange,14
6,Texas,Banana,40
7,Arizona,Banana,12
8,Florida,Banana,190


In [23]:
state_fruit2.melt()

,variable,value
0,State,Texas
1,State,Arizona
2,State,Florida
3,Apple,12
4,Apple,9
5,Apple,0
6,Orange,10
7,Orange,7
8,Orange,14
9,Banana,40


## <u>Stacking multiple groups of variables simultaneously</u>
<p>Some datasets have multiple groups of variables as column names that are stacked into their own columns. In this instance, we will tidy our actor DataFrame by simultaneously stacking the actor names and their corresponding Facebook likes with the <b>wide-to-long</b> function</p>

In [24]:
movie = pd.read_csv('data/movie.csv')

In [25]:
actor  = movie[['movie_title', 'actor_1_name',
                'actor_2_name', 'actor_3_name',
                'actor_1_facebook_likes',
                'actor_2_facebook_likes',
                'actor_3_facebook_likes']]

In [26]:
actor.head()

,movie_title,actor_1_name,actor_2_name,actor_3_name,actor_1_facebook_likes,actor_2_facebook_likes,actor_3_facebook_likes
0,Avatar,CCH Pounder,Joel David Moore,Wes Studi,1000.0,936.0,855.0
1,Pirates of the Caribbean: At World's End,Johnny Depp,Orlando Bloom,Jack Davenport,40000.0,5000.0,1000.0
2,Spectre,Christoph Waltz,Rory Kinnear,Stephanie Sigman,11000.0,393.0,161.0
3,The Dark Knight Rises,Tom Hardy,Christian Bale,Joseph Gordon-Levitt,27000.0,23000.0,23000.0
4,Star Wars: Episode VII - The Force Awakens,Doug Walker,Rob Walker,NaN,131.0,12.0,NaN


In [98]:
actor.head()

,movie_title,actor_1_name,actor_2_name,actor_3_name,actor_1_facebook_likes,actor_2_facebook_likes,actor_3_facebook_likes
0,Avatar,CCH Pounder,Joel David Moore,Wes Studi,1000.0,936.0,855.0
1,Pirates of the Caribbean: At World's End,Johnny Depp,Orlando Bloom,Jack Davenport,40000.0,5000.0,1000.0
2,Spectre,Christoph Waltz,Rory Kinnear,Stephanie Sigman,11000.0,393.0,161.0
3,The Dark Knight Rises,Tom Hardy,Christian Bale,Joseph Gordon-Levitt,27000.0,23000.0,23000.0
4,Star Wars: Episode VII - The Force Awakens,Doug Walker,Rob Walker,NaN,131.0,12.0,NaN


In [104]:
actor1 = actor[['movie_title', 'actor_1_name', 'actor_2_name', 'actor_3_name']]

In [106]:
s1 = actor1.melt(id_vars = 'movie_title')

In [107]:
s1

,movie_title,variable,value
0,Avatar,actor_1_name,CCH Pounder
1,Pirates of the Caribbean: At World's End,actor_1_name,Johnny Depp
2,Spectre,actor_1_name,Christoph Waltz
3,The Dark Knight Rises,actor_1_name,Tom Hardy
4,Star Wars: Episode VII - The Force Awakens,actor_1_name,Doug Walker
...,...,...,...
14743,Signed Sealed Delivered,actor_3_name,Crystal Lowe
14744,The Following,actor_3_name,Sam Underwood
14745,A Plague So Pleasant,actor_3_name,David Chandler
14746,Shanghai Calling,actor_3_name,Eliza Coupe


In [110]:
actor2 = actor[['movie_title', 'actor_1_facebook_likes', 'actor_2_facebook_likes', 'actor_3_facebook_likes']]

In [111]:
actor2

,movie_title,actor_1_facebook_likes,actor_2_facebook_likes,actor_3_facebook_likes
0,Avatar,1000.0,936.0,855.0
1,Pirates of the Caribbean: At World's End,40000.0,5000.0,1000.0
2,Spectre,11000.0,393.0,161.0
3,The Dark Knight Rises,27000.0,23000.0,23000.0
4,Star Wars: Episode VII - The Force Awakens,131.0,12.0,NaN
...,...,...,...,...
4911,Signed Sealed Delivered,637.0,470.0,318.0
4912,The Following,841.0,593.0,319.0
4913,A Plague So Pleasant,0.0,0.0,0.0
4914,Shanghai Calling,946.0,719.0,489.0


In [112]:
s2 = actor2.melt(id_vars = 'movie_title')

In [113]:
s2

,movie_title,variable,value
0,Avatar,actor_1_facebook_likes,1000.0
1,Pirates of the Caribbean: At World's End,actor_1_facebook_likes,40000.0
2,Spectre,actor_1_facebook_likes,11000.0
3,The Dark Knight Rises,actor_1_facebook_likes,27000.0
4,Star Wars: Episode VII - The Force Awakens,actor_1_facebook_likes,131.0
...,...,...,...
14743,Signed Sealed Delivered,actor_3_facebook_likes,318.0
14744,The Following,actor_3_facebook_likes,319.0
14745,A Plague So Pleasant,actor_3_facebook_likes,0.0
14746,Shanghai Calling,actor_3_facebook_likes,489.0


In [114]:
s1

,movie_title,variable,value
0,Avatar,actor_1_name,CCH Pounder
1,Pirates of the Caribbean: At World's End,actor_1_name,Johnny Depp
2,Spectre,actor_1_name,Christoph Waltz
3,The Dark Knight Rises,actor_1_name,Tom Hardy
4,Star Wars: Episode VII - The Force Awakens,actor_1_name,Doug Walker
...,...,...,...
14743,Signed Sealed Delivered,actor_3_name,Crystal Lowe
14744,The Following,actor_3_name,Sam Underwood
14745,A Plague So Pleasant,actor_3_name,David Chandler
14746,Shanghai Calling,actor_3_name,Eliza Coupe


In [115]:
s1['movie_title'].equals(s2['movie_title'])

True

In [124]:
s3 = s2.copy()
rep1 = s3.replace({'actor_1_facebook_likes' : 1,
                   'actor_2_facebook_likes' : 2,
                   'actor_3_facebook_likes' : 3})

In [118]:
s4 = s1.copy()

In [119]:
s4.head()

,movie_title,variable,value
0,Avatar,actor_1_name,CCH Pounder
1,Pirates of the Caribbean: At World's End,actor_1_name,Johnny Depp
2,Spectre,actor_1_name,Christoph Waltz
3,The Dark Knight Rises,actor_1_name,Tom Hardy
4,Star Wars: Episode VII - The Force Awakens,actor_1_name,Doug Walker


In [123]:
s3

,movie_title,variable,value
0,Avatar,actor_1_facebook_likes,1000.0
1,Pirates of the Caribbean: At World's End,actor_1_facebook_likes,40000.0
2,Spectre,actor_1_facebook_likes,11000.0
3,The Dark Knight Rises,actor_1_facebook_likes,27000.0
4,Star Wars: Episode VII - The Force Awakens,actor_1_facebook_likes,131.0
...,...,...,...
14743,Signed Sealed Delivered,actor_3_facebook_likes,318.0
14744,The Following,actor_3_facebook_likes,319.0
14745,A Plague So Pleasant,actor_3_facebook_likes,0.0
14746,Shanghai Calling,actor_3_facebook_likes,489.0


In [125]:
rep2 = s4.replace({'actor_1_name' : 1,
                   'actor_2_name' : 2,
                   'actor_3_name' : 3})

In [127]:
rep1['variable'].equals(rep2['variable'])

True

In [128]:
rep1.head(3)

,movie_title,variable,value
0,Avatar,1,1000.0
1,Pirates of the Caribbean: At World's End,1,40000.0
2,Spectre,1,11000.0


In [129]:
rep2.head(3)

,movie_title,variable,value
0,Avatar,1,CCH Pounder
1,Pirates of the Caribbean: At World's End,1,Johnny Depp
2,Spectre,1,Christoph Waltz


In [ ]:
rep1 = 

In [27]:
#It may not be possible to do a simultaneous stacking of several stack of columns at once using stack/melt.
#We will use the wide_to_long function to do the simultaneous stacking
#The first step is to change the column names to be stacked to end in a digit:
def change_col_name(col_name):
    col_name = col_name.replace('_name', '')
    if 'facebook' in col_name:
        fb_idx = col_name.find('facebook')
        col_name = col_name[:5] + col_name[fb_idx - 1:] \
                                + col_name[5:fb_idx-1]
    return col_name

## <u>Inverting stacked data</u>
<p>The <b>stack</b> and  <b>melt</b> methods can be inverted using the <b>unstack</b> and <b>pivot</b> methods respectively. In this procedure, we will stack/melt a dataset and invert the operation to the original form using unstack/pivot</p>

In [28]:
#INSTNM will be set as the index. Undergraduate race columns will be used
college = pd.read_csv('data/college.csv', index_col = 'INSTNM')

In [29]:
college_filtered = college.filter(like = 'UGDS_')
college_filtered.head()

,UGDS_WHITE,UGDS_BLACK,UGDS_HISP,UGDS_ASIAN,UGDS_AIAN,UGDS_NHPI,UGDS_2MOR,UGDS_NRA,UGDS_UNKN
INSTNM,,,,,,,,,
Alabama A & M University,0.0333,0.9353,0.0055,0.0019,0.0024,0.0019,0.0000,0.0059,0.0138
University of Alabama at Birmingham,0.5922,0.2600,0.0283,0.0518,0.0022,0.0007,0.0368,0.0179,0.0100
Amridge University,0.2990,0.4192,0.0069,0.0034,0.0000,0.0000,0.0000,0.0000,0.2715
University of Alabama in Huntsville,0.6988,0.1255,0.0382,0.0376,0.0143,0.0002,0.0172,0.0332,0.0350
Alabama State University,0.0158,0.9208,0.0121,0.0019,0.0010,0.0006,0.0098,0.0243,0.0137


In [30]:
#An alternative for the above code is:
useful_col = lambda x: 'UGDS_' in x or x == 'INSTNM'
college2 = pd.read_csv('data/college.csv', index_col = 'INSTNM', usecols = useful_col)

In [31]:
college2.equals(college_filtered)

True

In [32]:
#Stack the data:
#Stack method drops missing values by default
#Ignoring the dropping of missing values will result in an equal value of the stacked and unstacked dataset
college_stacked = college2.stack(dropna = False)

In [33]:
college_stacked.head(18)

INSTNM                                         
Alabama A & M University             UGDS_WHITE    0.0333
                                     UGDS_BLACK    0.9353
                                     UGDS_HISP     0.0055
                                     UGDS_ASIAN    0.0019
                                     UGDS_AIAN     0.0024
                                     UGDS_NHPI     0.0019
                                     UGDS_2MOR     0.0000
                                     UGDS_NRA      0.0059
                                     UGDS_UNKN     0.0138
University of Alabama at Birmingham  UGDS_WHITE    0.5922
                                     UGDS_BLACK    0.2600
                                     UGDS_HISP     0.0283
                                     UGDS_ASIAN    0.0518
                                     UGDS_AIAN     0.0022
                                     UGDS_NHPI     0.0007
                                     UGDS_2MOR     0.0368
                        

In [34]:
#Returning it back to its original format:
unstacked_college = college_stacked.unstack()
unstacked_college.head()

,UGDS_WHITE,UGDS_BLACK,UGDS_HISP,UGDS_ASIAN,UGDS_AIAN,UGDS_NHPI,UGDS_2MOR,UGDS_NRA,UGDS_UNKN
INSTNM,,,,,,,,,
Alabama A & M University,0.0333,0.9353,0.0055,0.0019,0.0024,0.0019,0.0000,0.0059,0.0138
University of Alabama at Birmingham,0.5922,0.2600,0.0283,0.0518,0.0022,0.0007,0.0368,0.0179,0.0100
Amridge University,0.2990,0.4192,0.0069,0.0034,0.0000,0.0000,0.0000,0.0000,0.2715
University of Alabama in Huntsville,0.6988,0.1255,0.0382,0.0376,0.0143,0.0002,0.0172,0.0332,0.0350
Alabama State University,0.0158,0.9208,0.0121,0.0019,0.0010,0.0006,0.0098,0.0243,0.0137


In [35]:
#Confirmation:
unstacked_college.equals(college2)

True

In [36]:
#A similar procedure can be done with melt followed by pivot:
college3 = pd.read_csv('data/college.csv')

In [37]:
college3.head()

,INSTNM,CITY,STABBR,HBCU,MENONLY,WOMENONLY,RELAFFIL,SATVRMID,SATMTMID,DISTANCEONLY,...,UGDS_2MOR,UGDS_NRA,UGDS_UNKN,PPTUG_EF,CURROPER,PCTPELL,PCTFLOAN,UG25ABV,MD_EARN_WNE_P10,GRAD_DEBT_MDN_SUPP
0,Alabama A & M University,Normal,AL,1.0,0.0,0.0,0,424.0,420.0,0.0,...,0.0000,0.0059,0.0138,0.0656,1,0.7356,0.8284,0.1049,30300,33888
1,University of Alabama at Birmingham,Birmingham,AL,0.0,0.0,0.0,0,570.0,565.0,0.0,...,0.0368,0.0179,0.0100,0.2607,1,0.3460,0.5214,0.2422,39700,21941.5
2,Amridge University,Montgomery,AL,0.0,0.0,0.0,1,NaN,NaN,1.0,...,0.0000,0.0000,0.2715,0.4536,1,0.6801,0.7795,0.8540,40100,23370
3,University of Alabama in Huntsville,Huntsville,AL,0.0,0.0,0.0,0,595.0,590.0,0.0,...,0.0172,0.0332,0.0350,0.2146,1,0.3072,0.4596,0.2640,45500,24097
4,Alabama State University,Montgomery,AL,1.0,0.0,0.0,0,425.0,430.0,0.0,...,0.0098,0.0243,0.0137,0.0892,1,0.7347,0.7554,0.1270,26600,33118.5


In [38]:
col3_filtered = college3.filter(like = 'UGDS_')
col3_filtered.head()

,UGDS_WHITE,UGDS_BLACK,UGDS_HISP,UGDS_ASIAN,UGDS_AIAN,UGDS_NHPI,UGDS_2MOR,UGDS_NRA,UGDS_UNKN
0,0.0333,0.9353,0.0055,0.0019,0.0024,0.0019,0.0000,0.0059,0.0138
1,0.5922,0.2600,0.0283,0.0518,0.0022,0.0007,0.0368,0.0179,0.0100
2,0.2990,0.4192,0.0069,0.0034,0.0000,0.0000,0.0000,0.0000,0.2715
3,0.6988,0.1255,0.0382,0.0376,0.0143,0.0002,0.0172,0.0332,0.0350
4,0.0158,0.9208,0.0121,0.0019,0.0010,0.0006,0.0098,0.0243,0.0137


In [39]:
col3_filtered['INSTNM'] = college3['INSTNM']
col3_filtered.head()

C:\Users\User\AppData\Local\Temp/ipykernel_13236/3424548193.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  col3_filtered['INSTNM'] = college3['INSTNM']


,UGDS_WHITE,UGDS_BLACK,UGDS_HISP,UGDS_ASIAN,UGDS_AIAN,UGDS_NHPI,UGDS_2MOR,UGDS_NRA,UGDS_UNKN,INSTNM
0,0.0333,0.9353,0.0055,0.0019,0.0024,0.0019,0.0000,0.0059,0.0138,Alabama A & M University
1,0.5922,0.2600,0.0283,0.0518,0.0022,0.0007,0.0368,0.0179,0.0100,University of Alabama at Birmingham
2,0.2990,0.4192,0.0069,0.0034,0.0000,0.0000,0.0000,0.0000,0.2715,Amridge University
3,0.6988,0.1255,0.0382,0.0376,0.0143,0.0002,0.0172,0.0332,0.0350,University of Alabama in Huntsville
4,0.0158,0.9208,0.0121,0.0019,0.0010,0.0006,0.0098,0.0243,0.0137,Alabama State University


In [40]:
#Alternatively:
college4 = pd.read_csv('data/college.csv', usecols = useful_col)

In [41]:
college4.head()

,INSTNM,UGDS_WHITE,UGDS_BLACK,UGDS_HISP,UGDS_ASIAN,UGDS_AIAN,UGDS_NHPI,UGDS_2MOR,UGDS_NRA,UGDS_UNKN
0,Alabama A & M University,0.0333,0.9353,0.0055,0.0019,0.0024,0.0019,0.0000,0.0059,0.0138
1,University of Alabama at Birmingham,0.5922,0.2600,0.0283,0.0518,0.0022,0.0007,0.0368,0.0179,0.0100
2,Amridge University,0.2990,0.4192,0.0069,0.0034,0.0000,0.0000,0.0000,0.0000,0.2715
3,University of Alabama in Huntsville,0.6988,0.1255,0.0382,0.0376,0.0143,0.0002,0.0172,0.0332,0.0350
4,Alabama State University,0.0158,0.9208,0.0121,0.0019,0.0010,0.0006,0.0098,0.0243,0.0137


In [42]:
college_melted = college4.melt(id_vars = 'INSTNM', 
                               var_name = 'Race',
                               value_name =  'Percentage')
college_melted.head()

,INSTNM,Race,Percentage
0,Alabama A & M University,UGDS_WHITE,0.0333
1,University of Alabama at Birmingham,UGDS_WHITE,0.5922
2,Amridge University,UGDS_WHITE,0.2990
3,University of Alabama in Huntsville,UGDS_WHITE,0.6988
4,Alabama State University,UGDS_WHITE,0.0158


In [43]:
#Inversion using pivot:
melted_inv = college_melted.pivot(index = 'INSTNM',
                                  columns = 'Race',
                                  values = 'Percentage')
melted_inv.head()

Race,UGDS_2MOR,UGDS_AIAN,UGDS_ASIAN,UGDS_BLACK,UGDS_HISP,UGDS_NHPI,UGDS_NRA,UGDS_UNKN,UGDS_WHITE
INSTNM,,,,,,,,,
A & W Healthcare Educators,0.0000,0.0,0.0000,0.9750,0.0250,0.0,0.0000,0.0000,0.0000
A T Still University of Health Sciences,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ABC Beauty Academy,0.0000,0.0,0.9333,0.0333,0.0333,0.0,0.0000,0.0000,0.0000
ABC Beauty College Inc,0.0000,0.0,0.0000,0.6579,0.0526,0.0,0.0000,0.0000,0.2895
AI Miami International University of Art and Design,0.0018,0.0,0.0018,0.0198,0.4773,0.0,0.0025,0.4644,0.0324


In [44]:
#In melt_inv, INSTNM is now the index & features are not in the initial order
#The .loc indexing is used to return to the initial ordering format
#It selects rows and columns simultaneously and then reset the index:
college2_replication = melted_inv.loc[college4['INSTNM'],
                                      college4.columns[1:]]\
                                 .reset_index()
college2_replication.head()

,INSTNM,UGDS_WHITE,UGDS_BLACK,UGDS_HISP,UGDS_ASIAN,UGDS_AIAN,UGDS_NHPI,UGDS_2MOR,UGDS_NRA,UGDS_UNKN
0,Alabama A & M University,0.0333,0.9353,0.0055,0.0019,0.0024,0.0019,0.0000,0.0059,0.0138
1,University of Alabama at Birmingham,0.5922,0.2600,0.0283,0.0518,0.0022,0.0007,0.0368,0.0179,0.0100
2,Amridge University,0.2990,0.4192,0.0069,0.0034,0.0000,0.0000,0.0000,0.0000,0.2715
3,University of Alabama in Huntsville,0.6988,0.1255,0.0382,0.0376,0.0143,0.0002,0.0172,0.0332,0.0350
4,Alabama State University,0.0158,0.9208,0.0121,0.0019,0.0010,0.0006,0.0098,0.0243,0.0137


In [45]:
college4.equals(college2_replication)

True

In [46]:
college2.head()

,UGDS_WHITE,UGDS_BLACK,UGDS_HISP,UGDS_ASIAN,UGDS_AIAN,UGDS_NHPI,UGDS_2MOR,UGDS_NRA,UGDS_UNKN
INSTNM,,,,,,,,,
Alabama A & M University,0.0333,0.9353,0.0055,0.0019,0.0024,0.0019,0.0000,0.0059,0.0138
University of Alabama at Birmingham,0.5922,0.2600,0.0283,0.0518,0.0022,0.0007,0.0368,0.0179,0.0100
Amridge University,0.2990,0.4192,0.0069,0.0034,0.0000,0.0000,0.0000,0.0000,0.2715
University of Alabama in Huntsville,0.6988,0.1255,0.0382,0.0376,0.0143,0.0002,0.0172,0.0332,0.0350
Alabama State University,0.0158,0.9208,0.0121,0.0019,0.0010,0.0006,0.0098,0.0243,0.0137


In [47]:
#The behind the scenes of transposition
college2.stack().unstack(0).head()

INSTNM,Alabama A & M University,University of Alabama at Birmingham,Amridge University,University of Alabama in Huntsville,Alabama State University,The University of Alabama,Central Alabama Community College,Athens State University,Auburn University at Montgomery,Auburn University,...,MCI Institute of Technology-Boca Raton,West Coast University-Miami,National American University-Houston,Aparicio-Levy Technical College,Fred D. Learey Technical College,Hollywood Institute of Beauty Careers-West Palm Beach,Hollywood Institute of Beauty Careers-Casselberry,Coachella Valley Beauty College-Beaumont,Dewey University-Mayaguez,Coastal Pines Technical College
UGDS_WHITE,0.0333,0.5922,0.2990,0.6988,0.0158,0.7825,0.7255,0.7823,0.5328,0.8507,...,0.0199,0.1522,0.1858,0.2431,0.3731,0.2182,0.1200,0.3284,0.0,0.6762
UGDS_BLACK,0.9353,0.2600,0.4192,0.1255,0.9208,0.1119,0.2613,0.1200,0.3376,0.0704,...,0.2815,0.1739,0.6443,0.1215,0.1388,0.4182,0.3333,0.1045,0.0,0.2508
UGDS_HISP,0.0055,0.0283,0.0069,0.0382,0.0121,0.0348,0.0044,0.0191,0.0074,0.0248,...,0.6854,0.6087,0.0672,0.6243,0.3080,0.2364,0.4400,0.4925,1.0,0.0359
UGDS_ASIAN,0.0019,0.0518,0.0034,0.0376,0.0019,0.0106,0.0025,0.0053,0.0221,0.0227,...,0.0132,0.0217,0.0079,0.0055,0.0000,0.0182,0.0000,0.0149,0.0,0.0045
UGDS_AIAN,0.0024,0.0022,0.0000,0.0143,0.0010,0.0038,0.0044,0.0157,0.0044,0.0074,...,0.0000,0.0000,0.0079,0.0055,0.0000,0.0000,0.0000,0.0299,0.0,0.0034


In [48]:
college2.T

INSTNM,Alabama A & M University,University of Alabama at Birmingham,Amridge University,University of Alabama in Huntsville,Alabama State University,The University of Alabama,Central Alabama Community College,Athens State University,Auburn University at Montgomery,Auburn University,...,Strayer University-North Dallas,Strayer University-San Antonio,Strayer University-Stafford,WestMed College - Merced,Vantage College,SAE Institute of Technology San Francisco,Rasmussen College - Overland Park,National Personal Training Institute of Cleveland,Bay Area Medical Academy - San Jose Satellite Location,Excel Learning Center-San Antonio South
UGDS_WHITE,0.0333,0.5922,0.2990,0.6988,0.0158,0.7825,0.7255,0.7823,0.5328,0.8507,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
UGDS_BLACK,0.9353,0.2600,0.4192,0.1255,0.9208,0.1119,0.2613,0.1200,0.3376,0.0704,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
UGDS_HISP,0.0055,0.0283,0.0069,0.0382,0.0121,0.0348,0.0044,0.0191,0.0074,0.0248,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
UGDS_ASIAN,0.0019,0.0518,0.0034,0.0376,0.0019,0.0106,0.0025,0.0053,0.0221,0.0227,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
UGDS_AIAN,0.0024,0.0022,0.0000,0.0143,0.0010,0.0038,0.0044,0.0157,0.0044,0.0074,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
UGDS_NHPI,0.0019,0.0007,0.0000,0.0002,0.0006,0.0009,0.0000,0.0010,0.0016,0.0000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
UGDS_2MOR,0.0000,0.0368,0.0000,0.0172,0.0098,0.0261,0.0000,0.0174,0.0297,0.0000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
UGDS_NRA,0.0059,0.0179,0.0000,0.0332,0.0243,0.0268,0.0000,0.0057,0.0397,0.0100,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
UGDS_UNKN,0.0138,0.0100,0.2715,0.0350,0.0137,0.0026,0.0019,0.0334,0.0246,0.0140,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [49]:
#Alternatively:
college2.transpose()

INSTNM,Alabama A & M University,University of Alabama at Birmingham,Amridge University,University of Alabama in Huntsville,Alabama State University,The University of Alabama,Central Alabama Community College,Athens State University,Auburn University at Montgomery,Auburn University,...,Strayer University-North Dallas,Strayer University-San Antonio,Strayer University-Stafford,WestMed College - Merced,Vantage College,SAE Institute of Technology San Francisco,Rasmussen College - Overland Park,National Personal Training Institute of Cleveland,Bay Area Medical Academy - San Jose Satellite Location,Excel Learning Center-San Antonio South
UGDS_WHITE,0.0333,0.5922,0.2990,0.6988,0.0158,0.7825,0.7255,0.7823,0.5328,0.8507,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
UGDS_BLACK,0.9353,0.2600,0.4192,0.1255,0.9208,0.1119,0.2613,0.1200,0.3376,0.0704,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
UGDS_HISP,0.0055,0.0283,0.0069,0.0382,0.0121,0.0348,0.0044,0.0191,0.0074,0.0248,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
UGDS_ASIAN,0.0019,0.0518,0.0034,0.0376,0.0019,0.0106,0.0025,0.0053,0.0221,0.0227,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
UGDS_AIAN,0.0024,0.0022,0.0000,0.0143,0.0010,0.0038,0.0044,0.0157,0.0044,0.0074,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
UGDS_NHPI,0.0019,0.0007,0.0000,0.0002,0.0006,0.0009,0.0000,0.0010,0.0016,0.0000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
UGDS_2MOR,0.0000,0.0368,0.0000,0.0172,0.0098,0.0261,0.0000,0.0174,0.0297,0.0000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
UGDS_NRA,0.0059,0.0179,0.0000,0.0332,0.0243,0.0268,0.0000,0.0057,0.0397,0.0100,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
UGDS_UNKN,0.0138,0.0100,0.2715,0.0350,0.0137,0.0026,0.0019,0.0334,0.0246,0.0140,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## <u>Unstacking after a groupby aggregation</u>
<pThe <b>groupby()</b> method results in data that is easily consumable/understandable. Grouping by more than one column might
result in data that is not easily understandable. Since the index in groupby are the unique grouping columns, the 
<b>unstack</b> method can be useful in the rearrangement of data in a presentable manner thus ease of interpretation. In this 
procedure, we will use the employee dataset to perform an aggregation, grouping by multiple columns. We will then use unstack 
to reshape the results into an easily comparable format in different groups</p>

In [50]:
employee = pd.read_csv('data/employee.csv')
employee.head()

,UNIQUE_ID,POSITION_TITLE,DEPARTMENT,BASE_SALARY,RACE,EMPLOYMENT_TYPE,GENDER,EMPLOYMENT_STATUS,HIRE_DATE,JOB_DATE
0,0,ASSISTANT DIRECTOR (EX LVL),Municipal Courts Department,121862.0,Hispanic/Latino,Full Time,Female,Active,2006-06-12,2012-10-13
1,1,LIBRARY ASSISTANT,Library,26125.0,Hispanic/Latino,Full Time,Female,Active,2000-07-19,2010-09-18
2,2,POLICE OFFICER,Houston Police Department-HPD,45279.0,White,Full Time,Male,Active,2015-02-03,2015-02-03
3,3,ENGINEER/OPERATOR,Houston Fire Department (HFD),63166.0,White,Full Time,Male,Active,1982-02-08,1991-05-25
4,4,ELECTRICIAN,General Services Department,56347.0,White,Full Time,Male,Active,1989-06-19,1994-10-22


In [51]:
employee['POSITION_TITLE'].nunique()

330

In [52]:
employee['POSITION_TITLE'].value_counts()

SENIOR POLICE OFFICER              220
POLICE OFFICER                     184
FIRE FIGHTER                       138
POLICE SERGEANT                     98
ENGINEER/OPERATOR                   89
                                  ... 
ACCOUNT CLERK                        1
ABATEMENT SPECIALIST                 1
STUDENT INTERN II                    1
EVIDENCE TECHNICIAN                  1
BUILDING MAINTENANCE SUPERVISOR      1
Name: POSITION_TITLE, Length: 330, dtype: int64

In [53]:
#Finding the mean basic salary per race:
race_salary = employee.groupby('RACE')['BASE_SALARY'].mean().astype(int)

In [54]:
race_salary.shape

(6,)

In [55]:
race_salary.sort_values(ascending = False)

RACE
White                                64419
Asian/Pacific Islander               61660
American Indian or Alaskan Native    60272
Hispanic/Latino                      52345
Others                               51278
Black or African American            50137
Name: BASE_SALARY, dtype: int32

In [56]:
#The above groupby procedure does not require reshaping
#Let us find the average salary for all races by gender:
race_gender = employee.groupby(['RACE', 'GENDER'])['BASE_SALARY'].mean()
race_gender

RACE                               GENDER
American Indian or Alaskan Native  Female    60238.800000
                                   Male      60305.400000
Asian/Pacific Islander             Female    63226.300000
                                   Male      61033.906667
Black or African American          Female    48915.421233
                                   Male      51082.074074
Hispanic/Latino                    Female    46503.316176
                                   Male      54782.819018
Others                             Female    63785.000000
                                   Male      38771.000000
White                              Female    66793.352941
                                   Male      63940.388119
Name: BASE_SALARY, dtype: float64

In [57]:
#It is easier to compare male and female thus we will unstack by gender:
race_gender.unstack('GENDER')

GENDER,Female,Male
RACE,,
American Indian or Alaskan Native,60238.800000,60305.400000
Asian/Pacific Islander,63226.300000,61033.906667
Black or African American,48915.421233,51082.074074
Hispanic/Latino,46503.316176,54782.819018
Others,63785.000000,38771.000000
White,66793.352941,63940.388119


#We can unstack other columns as follows:
race_gender.unstack('RACE')

In [58]:
#Instances when multiple groupings result in a DataFrame:
agg2 = employee.groupby(['RACE', 'GENDER'])['BASE_SALARY']\
                  .agg(['mean', 'max', 'min'])\
                  .astype(int)

In [59]:
agg2

mean     max    min
RACE                              GENDER                      
American Indian or Alaskan Native Female  60238   98536  26125
                                  Male    60305   81239  26125
Asian/Pacific Islander            Female  63226  130416  26125
                                  Male    61033  163228  27914
Black or African American         Female  48915  150416  24960
                                  Male    51082  275000  26125
Hispanic/Latino                   Female  46503  126115  26125
                                  Male    54782  165216  26104
Others                            Female  63785   63785  63785
                                  Male    38771   38771  38771
White                             Female  66793  178331  27955
                                  Male    63940  210588  26125

In [60]:
#You can unstack according to specific parameters to achieve the output you need:
agg2.unstack('GENDER')

mean            max            min       
GENDER                            Female   Male  Female    Male Female   Male
RACE                                                                         
American Indian or Alaskan Native  60238  60305   98536   81239  26125  26125
Asian/Pacific Islander             63226  61033  130416  163228  26125  27914
Black or African American          48915  51082  150416  275000  24960  26125
Hispanic/Latino                    46503  54782  126115  165216  26125  26104
Others                             63785  38771   63785   38771  63785  38771
White                              66793  63940  178331  210588  27955  26125

## <u>Replicating pivot_table() with a groupby() aggregation</u>
<p>After a little massaging, it is possible to replicate its functionality with a groupby() aggregation. In this procedure, we
will use the flights dataset to create a pivot table and then recreate it using the groupby() operations. We will find the 
total number of cancelled flights per origin airport for each airline:</p>

In [61]:
flights = pd.read_csv('data/flights.csv')
flights.head()

,MONTH,DAY,WEEKDAY,AIRLINE,ORG_AIR,DEST_AIR,SCHED_DEP,DEP_DELAY,AIR_TIME,DIST,SCHED_ARR,ARR_DELAY,DIVERTED,CANCELLED
0,1,1,4,WN,LAX,SLC,1625,58.0,94.0,590,1905,65.0,0,0
1,1,1,4,UA,DEN,IAD,823,7.0,154.0,1452,1333,-13.0,0,0
2,1,1,4,MQ,DFW,VPS,1305,36.0,85.0,641,1453,35.0,0,0
3,1,1,4,AA,DFW,DCA,1555,7.0,126.0,1192,1935,-7.0,0,0
4,1,1,4,WN,LAX,MCI,1720,48.0,166.0,1363,2225,39.0,0,0


In [62]:
#The method aggfunc calculates mean of the values by default
#It works by the functionality of as pivot_table being an intersection of all unique combinations of grouping columns
fp = flights.pivot_table(index = 'AIRLINE',
                         columns = 'ORG_AIR',
                         values = 'CANCELLED',
                         aggfunc = 'sum',
                         fill_value = 0)\
            .round(2)

In [63]:
fp

ORG_AIR,ATL,DEN,DFW,IAH,LAS,LAX,MSP,ORD,PHX,SFO
AIRLINE,,,,,,,,,,
AA,3,4,86,3,3,11,3,35,4,2
AS,0,0,0,0,0,0,0,0,0,0
B6,0,0,0,0,0,0,0,0,0,1
DL,28,1,0,0,1,1,4,0,1,2
EV,18,6,27,36,0,0,6,53,0,0
F9,0,2,1,0,1,1,1,4,0,0
HA,0,0,0,0,0,0,0,0,0,0
MQ,5,0,62,0,0,0,0,85,0,0
NK,1,1,6,0,1,1,3,10,2,0


In [64]:
fp.shape

(14, 10)

In [65]:
#The groupby() function cannot do an exact replication 
#The solution is to group by all the columns in the index and column parameters first:
fg = flights.groupby(['AIRLINE', 'ORG_AIR'])['CANCELLED'].agg('sum')
fg.head()

AIRLINE  ORG_AIR
AA       ATL         3
         DEN         4
         DFW        86
         IAH         3
         LAS         3
Name: CANCELLED, dtype: int64

In [66]:
#Unstacking method will pivot the ORG_AIR index level to column names:
fg_unstack = fg.unstack('ORG_AIR', fill_value = 0)

In [67]:
fg_unstack

ORG_AIR,ATL,DEN,DFW,IAH,LAS,LAX,MSP,ORD,PHX,SFO
AIRLINE,,,,,,,,,,
AA,3,4,86,3,3,11,3,35,4,2
AS,0,0,0,0,0,0,0,0,0,0
B6,0,0,0,0,0,0,0,0,0,1
DL,28,1,0,0,1,1,4,0,1,2
EV,18,6,27,36,0,0,6,53,0,0
F9,0,2,1,0,1,1,1,4,0,0
HA,0,0,0,0,0,0,0,0,0,0
MQ,5,0,62,0,0,0,0,85,0,0
NK,1,1,6,0,1,1,3,10,2,0


In [68]:
fg_unstack.equals(fp)

True

In [69]:
#More complex pivot tables can be replicated with groupby:
fp2 = flights.pivot_table(index = ['AIRLINE', 'MONTH'],
                          columns = ['ORG_AIR', 'CANCELLED'],
                          values = ['DEP_DELAY', 'DIST'],
                          aggfunc = [np.sum, np.mean],
                          fill_value = 0
)

In [70]:
fp2

sum                                        ...  \
              DEP_DELAY                                        ...   
ORG_AIR             ATL      DEN      DFW     IAH      LAS     ...   
CANCELLED             0  1     0  1     0  1    0  1     0  1  ...   
AIRLINE MONTH                                                  ...   
AA      1           -13  0   113  0  4276 -3  117  0  1036  0  ...   
        2           -39  0    71  0  2662  0    8  0   -55  0  ...   
        3            -2  0    69  0  5692  0  109  0   326  0  ...   
        4             1  0   304  0  3518  0  104  0   790  0  ...   
        5            52  0   352  0  5510  0   55  0    93  0  ...   
...                 ... ..   ... ..   ... ..  ... ..   ... ..  ...   
WN      7          2604  0  1919  0     0  0    0  0  4600  0  ...   
        8          1718  0  1180  0     0  0    0  0  3151  0  ...   
        9          1033  0   705  0     0  0    0  0  1400  0  ...   
        11          700  0  1372  0     0  0    0  0  1309  0  ...   
        12         1679  0  1898  0     0  0    0  0  2327  0  ...   

                      mean                                              \
                      DIST                                               
ORG_AIR                LAX                       MSP               ORD   
CANCELLED                0            1            0    1            0   
AIRLINE MONTH                                                            
AA      1      1678.037037  2475.000000   809.000000  0.0  1068.876033   
        2      1745.892308  1818.000000  1008.000000  0.0  1193.782178   
        3      1781.567568  1744.000000   964.733333  0.0  1058.933333   
        4      1850.923913     0.000000   648.714286  0.0  1094.633094   
        5      1820.478261     0.000000   787.250000  0.0   998.774775   
...                    ...          ...          ...  ...          ...   
WN      7       912.453704   327.777778   647.266667  0.0     0.000000   
        8       835.404040   346.000000   508.703704  0.0     0.000000   
        9       830.210000   317.666667   644.416667  0.0     0.000000   
        11      748.404040   459.333333   573.642857  0.0     0.000000   
        12      811.695238  1067.000000   634.736842  0.0     0.000000   

                                                                    
                                                                    
ORG_AIR                            PHX                 SFO          
CANCELLED               1            0      1            0       1  
AIRLINE MONTH                                                       
AA      1        0.000000  1167.666667    0.0  1860.166667     0.0  
        2      771.142857  1311.461538  868.0  1337.916667  2586.0  
        3      802.000000  1171.363636    0.0  1502.758621     0.0  
        4      943.600000  1266.214286    0.0  1646.903226     0.0  
        5      999.500000  1240.444444    0.0  1436.892857     0.0  
...                   ...          ...    ...          ...     ...  
WN      7        0.000000   799.160256  369.0   636.210526     0.0  
        8        0.000000   891.569767    0.0   644.857143   392.0  
        9        0.000000   872.840000    0.0   731.578947   354.5  
        11       0.000000   823.258741  872.0   580.875000   392.0  
        12       0.000000   778.420690    0.0   782.256410     0.0  

[149 rows x 80 columns]

In [71]:
#More complex pivot tables can be replicated with groupby:
fg2 = flights.groupby(['AIRLINE', 'MONTH', 'ORG_AIR', 'CANCELLED'])\
                      ['DEP_DELAY', 'DIST']\
             .agg([np.sum, np.mean])\
             .unstack(['ORG_AIR', 'CANCELLED'], fill_value = 0)
fg2            

C:\Users\User\AppData\Local\Temp/ipykernel_13236/3711673343.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  fg2 = flights.groupby(['AIRLINE', 'MONTH', 'ORG_AIR', 'CANCELLED'])\


DEP_DELAY                                                     \
                    sum                                                      
ORG_AIR             ATL          DEN          DFW         IAH          LAS   
CANCELLED             0    1       0    1       0    1      0    1       0   
AIRLINE MONTH                                                                
AA      1         -13.0  0.0   113.0  0.0  4276.0 -3.0  117.0  0.0  1036.0   
        2         -39.0  0.0    71.0  0.0  2662.0  0.0    8.0  0.0   -55.0   
        3          -2.0  0.0    69.0  0.0  5692.0  0.0  109.0  0.0   326.0   
        4           1.0  0.0   304.0  0.0  3518.0  0.0  104.0  0.0   790.0   
        5          52.0  0.0   352.0  0.0  5510.0  0.0   55.0  0.0    93.0   
...                 ...  ...     ...  ...     ...  ...    ...  ...     ...   
WN      7        2604.0  0.0  1919.0  0.0     0.0  0.0    0.0  0.0  4600.0   
        8        1718.0  0.0  1180.0  0.0     0.0  0.0    0.0  0.0  3151.0   
        9        1033.0  0.0   705.0  0.0     0.0  0.0    0.0  0.0  1400.0   
        11        700.0  0.0  1372.0  0.0     0.0  0.0    0.0  0.0  1309.0   
        12       1679.0  0.0  1898.0  0.0     0.0  0.0    0.0  0.0  2327.0   

                    ...         DIST                                 \
                    ...         mean                                  
ORG_AIR             ...          LAX                       MSP        
CANCELLED        1  ...            0            1            0    1   
AIRLINE MONTH       ...                                               
AA      1      0.0  ...  1678.037037  2475.000000   809.000000  0.0   
        2      0.0  ...  1745.892308  1818.000000  1008.000000  0.0   
        3      0.0  ...  1781.567568  1744.000000   964.733333  0.0   
        4      0.0  ...  1850.923913     0.000000   648.714286  0.0   
        5      0.0  ...  1820.478261     0.000000   787.250000  0.0   
...            ...  ...          ...          ...          ...  ...   
WN      7      0.0  ...   912.453704   327.777778   647.266667  0.0   
        8      0.0  ...   835.404040   346.000000   508.703704  0.0   
        9      0.0  ...   830.210000   317.666667   644.416667  0.0   
        11     0.0  ...   748.404040   459.333333   573.642857  0.0   
        12     0.0  ...   811.695238  1067.000000   634.736842  0.0   

                                                                         \
                                                                          
ORG_AIR                ORD                      PHX                 SFO   
CANCELLED                0           1            0      1            0   
AIRLINE MONTH                                                             
AA      1      1068.876033    0.000000  1167.666667    0.0  1860.166667   
        2      1193.782178  771.142857  1311.461538  868.0  1337.916667   
        3      1058.933333  802.000000  1171.363636    0.0  1502.758621   
        4      1094.633094  943.600000  1266.214286    0.0  1646.903226   
        5       998.774775  999.500000  1240.444444    0.0  1436.892857   
...                    ...         ...          ...    ...          ...   
WN      7         0.000000    0.000000   799.160256  369.0   636.210526   
        8         0.000000    0.000000   891.569767    0.0   644.857143   
        9         0.000000    0.000000   872.840000    0.0   731.578947   
        11        0.000000    0.000000   823.258741  872.0   580.875000   
        12        0.000000    0.000000   778.420690    0.0   782.256410   

                       
                       
ORG_AIR                
CANCELLED           1  
AIRLINE MONTH          
AA      1         0.0  
        2      2586.0  
        3         0.0  
        4         0.0  
        5         0.0  
...               ...  
WN      7         0.0  
        8       392.0  
        9       354.5  
        11      392.0  
        12        0.0  

[149 rows x 80 columns]

In [72]:
#swaplevels changes the order of the two first columns
#You can also cancel the levels of the other collumns
#Similarly for the rows, it can also be done but axis will be specified as 0
fg3 = flights.groupby(['AIRLINE', 'MONTH', 'ORG_AIR', 'CANCELLED'])\
                      ['DEP_DELAY', 'DIST']\
             .agg([np.sum, np.mean])\
             .unstack(['ORG_AIR', 'CANCELLED'], fill_value = 0)\
             .swaplevel(0, 1, axis = 'columns')
fg3  

C:\Users\User\AppData\Local\Temp/ipykernel_13236/4159623079.py:4: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  fg3 = flights.groupby(['AIRLINE', 'MONTH', 'ORG_AIR', 'CANCELLED'])\


sum                                                     \
              DEP_DELAY                                                      
ORG_AIR             ATL          DEN          DFW         IAH          LAS   
CANCELLED             0    1       0    1       0    1      0    1       0   
AIRLINE MONTH                                                                
AA      1         -13.0  0.0   113.0  0.0  4276.0 -3.0  117.0  0.0  1036.0   
        2         -39.0  0.0    71.0  0.0  2662.0  0.0    8.0  0.0   -55.0   
        3          -2.0  0.0    69.0  0.0  5692.0  0.0  109.0  0.0   326.0   
        4           1.0  0.0   304.0  0.0  3518.0  0.0  104.0  0.0   790.0   
        5          52.0  0.0   352.0  0.0  5510.0  0.0   55.0  0.0    93.0   
...                 ...  ...     ...  ...     ...  ...    ...  ...     ...   
WN      7        2604.0  0.0  1919.0  0.0     0.0  0.0    0.0  0.0  4600.0   
        8        1718.0  0.0  1180.0  0.0     0.0  0.0    0.0  0.0  3151.0   
        9        1033.0  0.0   705.0  0.0     0.0  0.0    0.0  0.0  1400.0   
        11        700.0  0.0  1372.0  0.0     0.0  0.0    0.0  0.0  1309.0   
        12       1679.0  0.0  1898.0  0.0     0.0  0.0    0.0  0.0  2327.0   

                    ...         mean                                 \
                    ...         DIST                                  
ORG_AIR             ...          LAX                       MSP        
CANCELLED        1  ...            0            1            0    1   
AIRLINE MONTH       ...                                               
AA      1      0.0  ...  1678.037037  2475.000000   809.000000  0.0   
        2      0.0  ...  1745.892308  1818.000000  1008.000000  0.0   
        3      0.0  ...  1781.567568  1744.000000   964.733333  0.0   
        4      0.0  ...  1850.923913     0.000000   648.714286  0.0   
        5      0.0  ...  1820.478261     0.000000   787.250000  0.0   
...            ...  ...          ...          ...          ...  ...   
WN      7      0.0  ...   912.453704   327.777778   647.266667  0.0   
        8      0.0  ...   835.404040   346.000000   508.703704  0.0   
        9      0.0  ...   830.210000   317.666667   644.416667  0.0   
        11     0.0  ...   748.404040   459.333333   573.642857  0.0   
        12     0.0  ...   811.695238  1067.000000   634.736842  0.0   

                                                                         \
                                                                          
ORG_AIR                ORD                      PHX                 SFO   
CANCELLED                0           1            0      1            0   
AIRLINE MONTH                                                             
AA      1      1068.876033    0.000000  1167.666667    0.0  1860.166667   
        2      1193.782178  771.142857  1311.461538  868.0  1337.916667   
        3      1058.933333  802.000000  1171.363636    0.0  1502.758621   
        4      1094.633094  943.600000  1266.214286    0.0  1646.903226   
        5       998.774775  999.500000  1240.444444    0.0  1436.892857   
...                    ...         ...          ...    ...          ...   
WN      7         0.000000    0.000000   799.160256  369.0   636.210526   
        8         0.000000    0.000000   891.569767    0.0   644.857143   
        9         0.000000    0.000000   872.840000    0.0   731.578947   
        11        0.000000    0.000000   823.258741  872.0   580.875000   
        12        0.000000    0.000000   778.420690    0.0   782.256410   

                       
                       
ORG_AIR                
CANCELLED           1  
AIRLINE MONTH          
AA      1         0.0  
        2      2586.0  
        3         0.0  
        4         0.0  
        5         0.0  
...               ...  
WN      7         0.0  
        8       392.0  
        9       354.5  
        11      392.0  
        12        0.0  

[149 rows x 80 columns]

In [73]:
fg3.ndim

2

In [74]:
fg3.size

11920

In [75]:
fg3.shape

(149, 80)

## <u>Renaming axis levels for easy reshaping</u>
<p>The reshaping process during the use of stack and unstack methods can come along with a name on the axes. Explicit is better
than implicit according to The Zen of Python. In this procedure, we will name each level of each axis and then use the stack 
and unstack methods to reshape the data to the desired form. We will use the college dataset then get some statistics done on 
it with consideration to the undergraduate population and the SAT math scores by institutions and religious affiliations:</p>

In [76]:
college.head()

,CITY,STABBR,HBCU,MENONLY,WOMENONLY,RELAFFIL,SATVRMID,SATMTMID,DISTANCEONLY,UGDS,...,UGDS_2MOR,UGDS_NRA,UGDS_UNKN,PPTUG_EF,CURROPER,PCTPELL,PCTFLOAN,UG25ABV,MD_EARN_WNE_P10,GRAD_DEBT_MDN_SUPP
INSTNM,,,,,,,,,,,,,,,,,,,,,
Alabama A & M University,Normal,AL,1.0,0.0,0.0,0,424.0,420.0,0.0,4206.0,...,0.0000,0.0059,0.0138,0.0656,1,0.7356,0.8284,0.1049,30300,33888
University of Alabama at Birmingham,Birmingham,AL,0.0,0.0,0.0,0,570.0,565.0,0.0,11383.0,...,0.0368,0.0179,0.0100,0.2607,1,0.3460,0.5214,0.2422,39700,21941.5
Amridge University,Montgomery,AL,0.0,0.0,0.0,1,NaN,NaN,1.0,291.0,...,0.0000,0.0000,0.2715,0.4536,1,0.6801,0.7795,0.8540,40100,23370
University of Alabama in Huntsville,Huntsville,AL,0.0,0.0,0.0,0,595.0,590.0,0.0,5451.0,...,0.0172,0.0332,0.0350,0.2146,1,0.3072,0.4596,0.2640,45500,24097
Alabama State University,Montgomery,AL,1.0,0.0,0.0,0,425.0,430.0,0.0,4811.0,...,0.0098,0.0243,0.0137,0.0892,1,0.7347,0.7554,0.1270,26600,33118.5


In [77]:
cg = college.groupby(['STABBR', 'RELAFFIL'])\
                     ['UGDS', 'SATMTMID']\
            .agg(['size', 'min', 'max'])\
            .head(10)

C:\Users\User\AppData\Local\Temp/ipykernel_13236/3056678374.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  cg = college.groupby(['STABBR', 'RELAFFIL'])\


In [78]:
cg

UGDS                   SATMTMID              
                size     min       max     size    min    max
STABBR RELAFFIL                                              
AK     0           7   109.0   12865.0        7    NaN    NaN
       1           3    27.0     275.0        3  503.0  503.0
AL     0          72    12.0   29851.0       72  420.0  590.0
       1          24    13.0    3033.0       24  400.0  560.0
AR     0          68    18.0   21405.0       68  427.0  565.0
       1          18    20.0    4485.0       18  495.0  600.0
AS     0           1  1276.0    1276.0        1    NaN    NaN
AZ     0         124     1.0  151558.0      124  503.0  580.0
       1           9    25.0    4102.0        9  480.0  480.0
CA     0         609     0.0   44744.0      609  445.0  785.0

In [79]:
#Renaming the first two columns:
cg = cg.rename_axis(['AGG_COLS', 'AGG_FUNCS'], axis = 'columns')

In [80]:
cg.head(10)

AGG_COLS        UGDS                   SATMTMID              
AGG_FUNCS       size     min       max     size    min    max
STABBR RELAFFIL                                              
AK     0           7   109.0   12865.0        7    NaN    NaN
       1           3    27.0     275.0        3  503.0  503.0
AL     0          72    12.0   29851.0       72  420.0  590.0
       1          24    13.0    3033.0       24  400.0  560.0
AR     0          68    18.0   21405.0       68  427.0  565.0
       1          18    20.0    4485.0       18  495.0  600.0
AS     0           1  1276.0    1276.0        1    NaN    NaN
AZ     0         124     1.0  151558.0      124  503.0  580.0
       1           9    25.0    4102.0        9  480.0  480.0
CA     0         609     0.0   44744.0      609  445.0  785.0

In [81]:
#Use of stack to move AGG_FUNCS to the index level:
cg.stack('AGG_FUNCS')

AGG_COLS                       UGDS  SATMTMID
STABBR RELAFFIL AGG_FUNCS                    
AK     0        size            7.0       7.0
                min           109.0       NaN
                max         12865.0       NaN
       1        size            3.0       3.0
                min            27.0     503.0
                max           275.0     503.0
AL     0        size           72.0      72.0
                min            12.0     420.0
                max         29851.0     590.0
       1        size           24.0      24.0
                min            13.0     400.0
                max          3033.0     560.0
AR     0        size           68.0      68.0
                min            18.0     427.0
                max         21405.0     565.0
       1        size           18.0      18.0
                min            20.0     495.0
                max          4485.0     600.0
AS     0        size            1.0       1.0
                min          1276.0       NaN
                max          1276.0       NaN
AZ     0        size          124.0     124.0
                min             1.0     503.0
                max        151558.0     580.0
       1        size            9.0       9.0
                min            25.0     480.0
                max          4102.0     480.0
CA     0        size          609.0     609.0
                min             0.0     445.0
                max         44744.0     785.0

In [82]:
cg.stack('AGG_FUNCS').swaplevel('AGG_FUNCS', 'STABBR', axis = 'index')

,,AGG_COLS,UGDS,SATMTMID
AGG_FUNCS,RELAFFIL,STABBR,,
size,0,AK,7.0,7.0
min,0,AK,109.0,NaN
max,0,AK,12865.0,NaN
size,1,AK,3.0,3.0
min,1,AK,27.0,503.0
max,1,AK,275.0,503.0
size,0,AL,72.0,72.0
min,0,AL,12.0,420.0
max,0,AL,29851.0,590.0


In [83]:
cg.stack('AGG_FUNCS')\
  .swaplevel('AGG_FUNCS', 'STABBR')\
  .sort_index(level = 'RELAFFIL', axis = 'index')\
  .sort_index(level = 'AGG_COLS', axis = 'columns')\
  .head(12)

AGG_COLS                   SATMTMID      UGDS
AGG_FUNCS RELAFFIL STABBR                    
max       0        AK           NaN   12865.0
                   AL         590.0   29851.0
                   AR         565.0   21405.0
                   AS           NaN    1276.0
                   AZ         580.0  151558.0
                   CA         785.0   44744.0
min       0        AK           NaN     109.0
                   AL         420.0      12.0
                   AR         427.0      18.0
                   AS           NaN    1276.0
                   AZ         503.0       1.0
                   CA         445.0       0.0

In [84]:
#To reshape yout data, you may need to chain the stacking and unstacking steps on the data:
cg.stack('AGG_FUNCS').unstack(['RELAFFIL', 'STABBR'])

AGG_COLS      UGDS                                                             \
RELAFFIL         0      1        0       1        0       1       0             
STABBR          AK     AK       AL      AL       AR      AR      AS        AZ   
AGG_FUNCS                                                                       
size           7.0    3.0     72.0    24.0     68.0    18.0     1.0     124.0   
min          109.0   27.0     12.0    13.0     18.0    20.0  1276.0       1.0   
max        12865.0  275.0  29851.0  3033.0  21405.0  4485.0  1276.0  151558.0   

AGG_COLS                   SATMTMID                                          \
RELAFFIL        1        0        0      1      0      1      0      1    0   
STABBR         AZ       CA       AK     AK     AL     AL     AR     AR   AS   
AGG_FUNCS                                                                     
size          9.0    609.0      7.0    3.0   72.0   24.0   68.0   18.0  1.0   
min          25.0      0.0      NaN  503.0  420.0  400.0  427.0  495.0  NaN   
max        4102.0  44744.0      NaN  503.0  590.0  560.0  565.0  600.0  NaN   

AGG_COLS                        
RELAFFIL              1      0  
STABBR        AZ     AZ     CA  
AGG_FUNCS                       
size       124.0    9.0  609.0  
min        503.0  480.0  445.0  
max        580.0  480.0  785.0

In [85]:
#You can stack all columns at once to return a Series:
cg.stack(['AGG_FUNCS', 'AGG_COLS']).head(12)

STABBR  RELAFFIL  AGG_FUNCS  AGG_COLS
AK      0         size       UGDS            7.0
                             SATMTMID        7.0
                  min        UGDS          109.0
                  max        UGDS        12865.0
        1         size       UGDS            3.0
                             SATMTMID        3.0
                  min        UGDS           27.0
                             SATMTMID      503.0
                  max        UGDS          275.0
                             SATMTMID      503.0
AL      0         size       UGDS           72.0
                             SATMTMID       72.0
dtype: float64

In [86]:
cg.stack(['AGG_FUNCS', 'AGG_COLS']).to_frame().head(12)

0
STABBR RELAFFIL AGG_FUNCS AGG_COLS         
AK     0        size      UGDS          7.0
                          SATMTMID      7.0
                min       UGDS        109.0
                max       UGDS      12865.0
       1        size      UGDS          3.0
                          SATMTMID      3.0
                min       UGDS         27.0
                          SATMTMID    503.0
                max       UGDS        275.0
                          SATMTMID    503.0
AL     0        size      UGDS         72.0
                          SATMTMID     72.0